# Inspecting Model Equations

One of the key features of ANDES is its symbolic-numeric hybrid framework, which allows you to inspect the mathematical equations that define each model. This capability is invaluable for understanding model behavior, debugging simulations, and verifying that models are implemented correctly.

Every model in ANDES is defined symbolically using SymPy, and the framework automatically generates numerical code for efficient simulation. This tutorial shows how to access and examine the symbolic equations, variables, and services that comprise each model.

## Setup

In [ ]:
import andes

andes.config_logger(30)  # Reduce logging verbosity

## Listing Available Models

To see all models supported by ANDES, create an empty System object and call `supported_models()`. Models are organized into groups based on their function in the power system.

In [ ]:
ss = andes.System()
print(ss.supported_models())

## Preparing Model Equations

Before inspecting a model's symbolic equations, you need to call `prepare()` on that model. This triggers the symbolic processing that generates the equations and Jacobian matrices. For a single model, this is fast; preparing all models takes longer but is still manageable.

In [ ]:
# Prepare equations for the GENCLS model (classical generator)
ss.GENCLS.prepare()

## Model Documentation

Each model provides comprehensive documentation through the `doc()` method. This returns a formatted string containing:
- Model description and group membership
- Parameter definitions with units and defaults
- Variable definitions (states and algebraics)
- Initialization equations
- Differential and algebraic equations
- Service definitions
- Configuration options

In [ ]:
print(ss.GENCLS.doc())

The documentation shows that GENCLS (classical generator) has two state variables (`delta` for rotor angle and `omega` for rotor speed) and several algebraic variables for currents, voltages, and power outputs. The differential equations implement the swing equation that governs generator dynamics.

## Accessing Variables

Model variables are stored in ordered dictionaries accessible through `states` and `algebs` attributes. This lets you enumerate all variables programmatically.

In [ ]:
# State (differential) variables
print("State variables:")
for name, var in ss.GENCLS.states.items():
    print(f"  {name}: {var}")

In [ ]:
# Algebraic variables
print("Algebraic variables:")
for name, var in ss.GENCLS.algebs.items():
    print(f"  {name}: {var}")

## Symbolic Equations

The symbolic equations are stored in `Model.syms`. This object contains SymPy expressions that can be displayed, manipulated, and analyzed. The main attributes are:

| Attribute | Contents |
|-----------|----------|
| `xy` | Vector of all variables (states then algebraics) |
| `f` | Differential equations (right-hand side of T x' = f) |
| `g` | Algebraic equations (right-hand side of 0 = g) |
| `df` | Jacobian df/dxy |
| `dg` | Jacobian dg/dxy |
| `s` | Service equations |

In [ ]:
# All variables in order
ss.GENCLS.syms.xy

### Differential Equations

The differential equations define how state variables evolve over time. For generators, these typically include the swing equation relating rotor angle and speed to mechanical and electrical torque.

In [ ]:
ss.GENCLS.syms.f

The first equation is the rotor angle dynamics: `d(delta)/dt = 2*pi*fn*(omega - 1)`, where `omega` is in per-unit of nominal frequency. The second equation is the swing equation: `M*d(omega)/dt = tm - te - D*(omega - 1)`, relating acceleration to the imbalance between mechanical torque `tm` and electrical torque `te`, with damping `D`.

### Algebraic Equations

Algebraic equations define relationships that must hold at every instant (they have no time derivative). These typically include current-voltage relationships, power calculations, and interface equations.

In [ ]:
ss.GENCLS.syms.g

### Jacobian Matrices

The Jacobian matrices contain partial derivatives of the equations with respect to variables. These are essential for Newton-Raphson iteration in power flow and implicit integration in time-domain simulation.

In [ ]:
# Jacobian of differential equations
ss.GENCLS.syms.df

In [ ]:
# Jacobian of algebraic equations (first 4 rows, first 6 columns)
ss.GENCLS.syms.dg[:4, :6]

## Services

Services are intermediate calculations that are computed once (at initialization) and reused throughout the simulation. They often represent derived quantities like initial values or converted parameters.

In [ ]:
# List all services
print("Services:")
for name, svc in ss.GENCLS.services.items():
    print(f"  {name}")

In [ ]:
# Service equations
ss.GENCLS.syms.s

## Practical Applications

Understanding model equations is useful for:

- **Verification**: Confirming that a model implements the expected physics
- **Debugging**: Identifying why a simulation produces unexpected results
- **Extension**: Understanding the structure before adding new features
- **Documentation**: Generating mathematical descriptions for papers or reports
- **Teaching**: Demonstrating power system dynamics concepts

## See Also

- {doc}`../modeling/concepts/framework-overview` - Overview of the symbolic-numeric framework
- {doc}`../modeling/components/variables` - Detailed variable type documentation
- {doc}`../modeling/creating-models/index` - Creating new models